<a href="https://colab.research.google.com/github/Disha-Sikka/Dog-Breed-Classification/blob/main/dogs_breed_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [15]:
!kaggle competitions download -c ssd-3-l-dog-breed-classification


ssd-3-l-dog-breed-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [16]:
import zipfile
zip_ref= zipfile.ZipFile('/content/ssd-3-l-dog-breed-classification.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [25]:
import os

BASE_DIR = "/content/dog_breed_competition"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")
CHECKPOINT_DIR = os.path.join(BASE_DIR, "checkpoints")
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

IMG_SIZE = 128
NUM_CLASSES = 157
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [26]:
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [27]:
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

val_test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])

train_ds = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
val_ds = datasets.ImageFolder(VAL_DIR, transform=val_test_transforms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

# Class mapping for predictions
idx_to_class = {v: k for k, v in train_ds.class_to_idx.items()}


In [28]:
class DogCNN(nn.Module):
    def __init__(self, num_classes=157):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d(2),  # 64x64

            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),  # 32x32

            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.MaxPool2d(2),  # 16x16

            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.AdaptiveAvgPool2d((4, 4))     # 4x4x256
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

model = DogCNN(num_classes=157).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)


In [30]:
train_losses, val_losses, val_accs = [], [], []
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_losses.append(running_loss / len(train_loader))

    # Validation
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            val_loss += criterion(outputs, labels).item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    val_losses.append(val_loss / len(val_loader))
    val_accs.append(val_acc)

    print(f"✅ Epoch {epoch+1}: Val Acc = {100*val_acc:.2f}%, Train Loss = {train_losses[-1]:.4f}")

    torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, f"model_epoch{epoch+1}.pth"))
    scheduler.step()


Epoch 1/10: 100%|██████████| 381/381 [01:02<00:00,  6.08it/s]


✅ Epoch 1: Val Acc = 1.38%, Train Loss = 4.5832


Epoch 2/10: 100%|██████████| 381/381 [01:02<00:00,  6.10it/s]


✅ Epoch 2: Val Acc = 0.96%, Train Loss = 4.4798


Epoch 3/10: 100%|██████████| 381/381 [01:02<00:00,  6.12it/s]


...

✅ Epoch 8: Val Acc = 2.52%, Train Loss = 4.2215


Epoch 9/10: 100%|██████████| 381/381 [01:04<00:00,  5.91it/s]


✅ Epoch 9: Val Acc = 2.22%, Train Loss = 4.1813


Epoch 10/10: 100%|██████████| 381/381 [01:02<00:00,  6.12it/s]


✅ Epoch 10: Val Acc = 3.10%, Train Loss = 4.1681
